<a href="https://colab.research.google.com/github/ayasherby/graduation_models/blob/main/Tooth_segmentation_using_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1- clone the github**

In [ ]:
!git clone https://github.com/Arnold0210/TEETH-RECOGNITION-WITH-MACHINE-LEARNING


Cloning into 'TEETH-RECOGNITION-WITH-MACHINE-LEARNING'...
remote: Enumerating objects: 32505, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 32505 (delta 19), reused 17 (delta 9), pack-reused 32477
Receiving objects: 100% (32505/32505), 6.29 GiB | 34.48 MiB/s, done.
Resolving deltas: 100% (5099/5099), done.
Updating files: 100% (14166/14166), done.


# **2-Install imprtant Libraries**

In [ ]:
!pip install opencv-python-headless numpy tqdm

In [ ]:
!pip install numpy pandas matplotlib scikit-learn

# **3-Change path of the runing**

In [ ]:
import os

# Change the working directory to your project root
project_root = '/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source'
os.chdir(project_root)

# Verify the change
print("Current Working Directory: ", os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory: '/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source'

# **4-Copy these cells and paste it in the moduels**

In [ ]:
#COPy this main.py

import errno
import os
import sys

import cv2 as cv
import numpy as np
from tqdm import tqdm

import Classification as Cl
import FeatureExtraction as fE
import PreProcessingData as pD
import ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset ORIGINAL
        # img, name = read.read_One_Image(self.PATH_IMAGES)
        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset RECORTADO
        img, name = read.read_One_Image(self.PATH_IMAGES)

        # Se obtiene las dimensiones de la imagen original
        height_ori, width_ori, depth_ori = img.shape
        # print("Image original shape: \n Height:", height_ori, ", Width:", width_ori)

        # Se realiza un ajuste de tamaño para reducir la imagen a unas dimensiones de 600x400
        img_resize = pp.resize_Image(img, name)

        # La imagen reajustada se convierte de BGR a RGB
        img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
        # Se convierte la imagen de RGB a HSV
        hsv_image = pp.rgb_2_HSV(img_resize, name)
        # Se saca la imagen en pila de tono de rojos
        stack, name = pp.stackColors(hsv_image, name)
        # Se saca los histogramas por imagen para determinar el rango de color de los dientes
        pp.hsv_hist(hsv_image, name)
        #
        # plt.imshow(img_resize)
        # Blur image slightly
        name, blurimage = pp.blurImage(img_resize, name)
        pp.show_mask(blurimage, name)
        pp.overlay_mask(blurimage, img_resize, name)

        # Se obtiene la rueda cromatica de la imágen
        # pp.getChromatiColor(img_resize,name,fe)

        # img_rgb2ycbcr = pp.rgb_2_YCrCb(img_resize, name)
        img_rgb2hsv = pp.rgb_2_HSV(img_resize, name)

    '''easygui.msgbox("Image original shape: \n Height:" + str(height_ori) + "px, Width:" + str(width_ori) + "px" +
                   "\n Image Resize shape: \n Height:" + str(height_res) + "px, Width:" + str(width_res) + "px",
                   image=os.path.join(os.path.join(os.getcwd(), os.path.pardir),
                                      'PreProcessing/ResizeImages/' + name),
                   title="Image Shape - PreProcessing ")'''

    def savebin(self):
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        images, names = read.read_Images(self.PATH_IMAGES_P)
        for image_point, name_point in zip(images, names):
            img_resize = pp.resize_Image(image_point, name_point)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_RGB2GRAY)
            pp.bin(img_resize, name_point)

    def main_alldataset(self):
        cc = self.clasification

        # Ask for user input for number of iterations and dataset splits
        times_execution = int(input('\nIndicate the number of iterations: '))
        folds = int(input('\nNumber of folds to split the dataset: '))
        test_size = float(input('\nPercentage of test dataset (e.g., 20 for 20%): '))

        for i in range(times_execution):
            # Read features and labels
            filefeaturespath = os.path.join(self.PROJECT_PATH, 'FeatureExtraction', 'features.csv')
            names, features = cc.readfeatures(filefeaturespath)
            labels = cc.readLabels(self.PATH_Labels)

            # Cross-validation
            X, Y = cc.CrossValidation(features, labels, test_size / 100)
            SVM_results, DT_results, KNN_results = cc.classification(
                self.PATH_IMAGES, X, Y, folds, tags=['0', '1', '2', '3'],
                target_names=['a1', 'a2', 'a3', 'a35'],
                vals_to_replace={'a1': '0', 'a2': '1', 'a3': '2', 'a35': '3'}
            )

            print(f'\n--------- Iteration {i+1} ---------')

            for SVM_result, DT_result, KNN_result in zip(SVM_results, DT_results, KNN_results):
                matrix_confusion_SVM, report_clasification_SVM, score_accuracy_SVM = SVM_result
                matrix_confusion_DT, report_clasification_DT, score_accuracy_DT = DT_result
                matrix_confusion_KNN, report_clasification_KNN, score_accuracy_KNN = KNN_result

                print('\n------- SVM Results -------')
                print('Confusion Matrix:\n', matrix_confusion_SVM)
                print('Classification Report:\n', report_clasification_SVM)
                print('Mean Accuracy:', np.mean(score_accuracy_SVM))

                print('\n------- Decision Tree Results -------')
                print('Confusion Matrix:\n', matrix_confusion_DT)
                print('Classification Report:\n', report_clasification_DT)
                print('Mean Accuracy:', np.mean(score_accuracy_DT))

                print('\n------- KNN Results -------')
                print('Confusion Matrix:\n', matrix_confusion_KNN)
                print('Classification Report:\n', report_clasification_KNN)
                print('Mean Accuracy:', np.mean(score_accuracy_KNN))

            print('--------- End of Iteration ---------')

if __name__ == '__main__':
    tesis = MainClass()
    tesis.main_alldataset()
    print('Experiment execution completed.')


In [ ]:
#Copy and paste to the paste it to the module
import errno
import os
from os import listdir
from os.path import isfile, join
from joblib import dump
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=155, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)

        k_folds = KFold(n_splits=n_splits)
        from sklearn import tree , neighbors
        SVM = []
        KNN =[]
        DT = []

        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            #index_images_name = [onlyfiles.index(element) for element in images_name if element in onlyfiles]
            index_images_name = [images_name.index(file.split('.')[0]) for file in onlyfiles if file.split('.')[0] in images_name]
            #index_images_name = []
            k_folds.get_n_splits(index_images_name)
            #print(list(images_name.index))

            for train_index, test_index in k_folds.split(index_images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                print("Training indices:", [index_images_name[i] for i in train_index])
                print("Testing indices:", [index_images_name[i] for i in test_index])

                for i in train_index:

                    currentFilename=images_name[i]
                    train_features.append(feature.to_numpy().tolist()[index_images_name[i]])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    train_label.append(labels_color[index_images_name[i]])

                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:

                    test_features.append(feature.to_numpy()[index_images_name[i]])
                    test_label.append(labels_color[index_images_name[i]])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)

            """for i in range(n_splits):
                svm.append(svm.SVC())
                knn.append(neighbors.KNeighborsClassifier())
                dt.append(tree.DecisionTreeClassifier())"""

            dump(SVM, os.path.join(self.path_project, 'svm_models.joblib'))
            dump(DT, os.path.join(self.path_project, 'dt_models.joblib'))
            dump(KNN, os.path.join(self.path_project, 'knn_models.joblib'))
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
# copy and past to FeatureExtraction.py
import errno
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.cluster import KMeans

import PreProcessingData as pD
import ReadImages as rI


class FeatureExtraction:
    readImages = None
    preProcessing = None

    def __init__(self, PATH_PROJECT, PATH_IMAGES):
        self.path_project = os.path.join(PATH_PROJECT, 'FeatureExtraction')
        self.path_dataset = PATH_IMAGES
        try:
            if not os.path.exists(self.path_project + 'GetColors'):
                self.path_getColor = os.path.join(self.path_project, 'GetColors')
                os.mkdir(self.path_getColor)

                print('ResizeImages Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('ResizeImages Directory Already Exists.')
            else:
                raise
        self.preProcessing = pD.PreProcessingData(PATH_PROJECT, PATH_IMAGES)
        self.readImages = rI.LoadData(PATH_PROJECT)

    def RGB2HEX(self, color):
        return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

    def get_colors(self, src, number_of_colors, show_chart, name):
        modified_image = src.reshape(src.shape[0] * src.shape[1], 3)
        clf = KMeans(n_clusters=number_of_colors)
        labels = clf.fit_predict(modified_image)
        counts = Counter(labels)
        center_colors = clf.cluster_centers_
        # We get ordered colors by iterating through the keys
        ordered_colors = [center_colors[i] for i in counts.keys()]
        hex_colors = [self.RGB2HEX(ordered_colors[i]) for i in counts.keys()]
        rgb_colors = [ordered_colors[i] for i in counts.keys()]

        if show_chart:
            path_image_name = os.path.join(self.path_getColor, name)
            plt.figure(figsize=(8, 6))
            plt.pie(counts.values(), labels=hex_colors, colors=hex_colors)
            plt.savefig(path_image_name)
            plt.show()

        return rgb_colors, hex_colors

    def getFeaturesVector(self, image, mask):
        features = []
        imagecpy = image.copy()
        for i in range(len(mask)):
            for j in range(len(mask[i])):
                if (j == 255):
                    # print(image[i][j])
                    features.append(imagecpy[i][j])
        return features

    def meanVector(self, vector_caracteristicas):
        return np.mean(vector_caracteristicas)

    def varVector(self, vector_caracteristicas):
        return stats.moment(vector_caracteristicas)

    def skewVector(self, vector_caracteristicas):
        return stats.skew(vector_caracteristicas)

    def getFeatures(self, imagen, filefeaturespath, name_point):
        if not (os.path.exists(filefeaturespath) or os.path.isfile(filefeaturespath)):
            filefeatures = open(filefeaturespath, "w")
        else:
            filefeatures = open(filefeaturespath, "a")
        features = []
        for j in range(0, len(imagen)):
            features.append(self.meanVector(imagen[j]))
            features.append(self.varVector(imagen[j]))
            features.append(self.skewVector(imagen[j]))
        filefeatures.write(name_point)
        for item in range(len(features)):
            filefeatures.write(",%.6f" % features[item])
        filefeatures.write("\n")#"," + label +'''
        filefeatures.close()


In [ ]:
#Copy it and paste it into PRePRocessing.py

import errno
import os

import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt, colors
#from pygments.util import xrange


class PreProcessingData:
    path_resize = None
    path_RGB2YCrCb = None
    path_RGB2HSV = None
    path_RGB2Lab = None
    path_RGB2LAB = None
    path_segmentation = None
    path_BarPlot = None
    path_BarPlotCh1 = None
    path_BarPlotCh2 = None
    path_BarPlotCh3 = None
    path_Mask = None
    path_Mask_Overlay = None
    path_Inverse = None
    path_project = None
    path_dataset = None

    def __init__(self, PATH_PROJECT, PATH_DATASET):

        self.path_project = os.path.join(PATH_PROJECT, 'PreProcessing')
        self.path_dataset = PATH_DATASET
        try:
            if not os.path.exists(self.path_project + 'Segmentation'):
                self.path_segmentation = os.path.join(self.path_project, 'Segmentation')
                os.mkdir(self.path_segmentation)
                print('Segmentation Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Segmentation Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'BarPlot'):
                self.path_BarPlot = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'BarPlot')

                os.mkdir(self.path_BarPlot)
                print('BarPlot Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('BarPlot Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'BarPlotCh1'):
                self.path_BarPlotCh1 = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'BarPlotCh1')

                os.mkdir(self.path_BarPlotCh1)
                print('BarPlotCh1 Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('BarPlotCh1 Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'BarPlotCh2'):
                self.path_BarPlotCh2 = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'BarPlotCh2')

                os.mkdir(self.path_BarPlotCh2)
                print('BarPlotCh2 Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('BarPlotCh2 Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'BarPlotCh3'):
                self.path_BarPlotCh3 = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'BarPlotCh3')

                os.mkdir(self.path_BarPlotCh3)
                print('BarPlotCh3 Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('BarPlotCh3 Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'Mask'):
                self.path_Mask = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'Mask')

                os.mkdir(self.path_Mask)
                print('Mask Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('Mask Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'MaskOverlay'):
                self.path_Mask_Overlay = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'MaskOverlay')

                os.mkdir(self.path_Mask_Overlay)
                print('MaskOverlay Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('MaskOverlay Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(os.path.join(self.path_project, 'Segmentation') + 'Inverse'):
                self.path_Inverse = os.path.join(os.path.join(self.path_project, 'Segmentation'), 'Inverse')

                os.mkdir(self.path_Inverse)
                print('Inverse Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('Inverse Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.path_project + 'ResizeImages'):
                self.path_resize = os.path.join(self.path_project, 'ResizeImages')
                os.mkdir(self.path_resize)
                print('ResizeImages Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('ResizeImages Directory Already Exists.')
            else:
                raise

        try:
            if not os.path.exists(self.path_project + 'RGB2YCrCb'):
                self.path_RGB2YCrCb = os.path.join(self.path_project, 'RGB2YCrCb')
                os.mkdir(self.path_RGB2YCrCb)
                print('RGB2YCrCb Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('RGB2YCbCr Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.path_project + 'HSV'):
                self.path_RGB2HSV = os.path.join(self.path_project, 'HSV')
                os.mkdir(self.path_RGB2HSV)
                print('HSV Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('HSV Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.path_project + 'LAB'):
                self.path_RGB2LAB = os.path.join(self.path_project, 'LAB')
                os.mkdir(self.path_RGB2LAB)
                print('LAB Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('LAB Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.path_project + 'Lab_'):
                self.path_RGB2Lab = os.path.join(self.path_project, 'Lab_')
                os.mkdir(self.path_RGB2Lab)
                print('Lab_ Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:

                print('Lab_ Directory Already Exists.')
            else:
                raise

    def bin(self, img, name):
        path_bin = os.path.join(self.path_Inverse, name)
        ret, thresh1 = cv.threshold(img, 127, 255, cv.THRESH_BINARY)
        ret, thresh2 = cv.threshold(img, 127, 255, cv.THRESH_BINARY_INV)
        ret, thresh3 = cv.threshold(img, 127, 255, cv.THRESH_TRUNC)
        ret, thresh4 = cv.threshold(img, 127, 255, cv.THRESH_TOZERO)
        ret, thresh5 = cv.threshold(img, 127, 255, cv.THRESH_TOZERO_INV)
        thresh6 = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 37, 2)
        # titles = ['Original Image', 'BINARY', 'BINARY_INV', 'TRUNC', 'TOZERO', 'TOZERO_INV']
        titles = ['Original Image', 'BINARY', 'BINARY_INV', 'TRUNC', 'TOZERO', 'ADAPTATIVE']
        # images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]
        images = [img, thresh1, thresh2, thresh3, thresh4, thresh6]
        for i in xrange(6):
            plt.subplot(2, 3, i + 1), plt.imshow(images[i], 'gray')
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
        plt.savefig(path_bin)
        plt.close('all')

    def resize_Image(self, image, name):
        img = cv.resize(image, dsize=None, fx=1 / 3, fy=1 / 3, interpolation=cv.INTER_LINEAR)
        # img = cv.resize(src=image,None, fx=1 / 3, fy=1 / 3)
        path_name_image = os.path.join(self.path_resize, name)
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    def rgb_2_YCrCb(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_RGB2YCR_CB)
        path_name_image = os.path.join(self.path_RGB2YCrCb, name)
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    def rgb_2_HSV(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_RGB2HSV_FULL)
        path_name_image = os.path.join(self.path_RGB2HSV, name)
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    '''def rgb_2_HSI(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_HS)
        path_name_image = os.path.join(self.path_RGB2HSI, name)
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img'''

    def rgb_2_LAB(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_RGB2LAB)
        path_name_image = os.path.join(self.path_RGB2LAB, name)
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    def rgb_2_Lab(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_RGB2Lab)
        path_name_image = os.path.join(self.path_RGB2Lab, name)
        print('Exitoso RGB2_LAB')
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    def bgr_2_Lab(self, image, name):
        img = cv.cvtColor(image, cv.COLOR_RGB2Lab)
        path_name_image = os.path.join(self.path_RGB2Lab, name)
        print('Exitoso RGB2_LAB')
        if os.path.exists(path_name_image):
            pass
        else:
            cv.imwrite(path_name_image, img)
        return img

    def getChromatiColor(self, image, name, fe):
        plt.savefig(
            os.path.join(os.path.join(os.getcwd(), os.path.pardir), 'FeatureExtraction/GetColors/Original' + name))
        plt.close('all')
        height_res, width_res, depth_res = image.shape
        print("name", name)
        print("Image Resize shape: \n Height:", height_res, ", Width:", width_res)
        rgbcolors, hexcolors = fe.get_colors(image, 15, True, 'plot_' + name)
        print("RGB:\n", rgbcolors, "\n", "hexcolors:\n", hexcolors)

    def stackColors(self, image, name):
        namefoldersplit = str.split(name, '.')
        namefolder = namefoldersplit[0]
        images = []
        for i in [0, 1, 2]:
            colour = image.copy()
            if i != 0: colour[:, :, 0] = 0
            if i != 1: colour[:, :, 1] = 255
            if i != 2: colour[:, :, 2] = 255
            images.append(colour)
        hsv_stack = np.vstack(images)
        rgb_stack = cv.cvtColor(hsv_stack, cv.COLOR_HSV2RGB)
        plt.imshow(rgb_stack)
        plt.savefig(self.path_BarPlot + '\\BAR_StackColor_' + name)
        plt.close('all')
        return rgb_stack, name

    def hsv_hist(self, image, name):
        namefoldersplit = str.split(name, '.')
        namefolder = namefoldersplit[0]
        # Hue
        plt.figure(figsize=(20, 3))
        histr = cv.calcHist([image], [0], None, [180], [0, 180])
        plt.xlim([0, 180])
        colours = [colors.hsv_to_rgb((i / 180, 1, 0.9)) for i in range(0, 180)]
        colours = np.array(colours)
        histr = np.array(histr)
        # print(range(0,180))
        x = list(range(0, 180))
        # print(histr.shape)
        # print(len(x))
        # print(colours.shape)
        for i in x:
            plt.bar(x[i], histr[i], color=colours[i], edgecolor=colours[i], width=1)
            plt.title('Hue')
        plt.savefig(self.path_BarPlotCh1 + '\\BAR_CHART_HUE_' + name)

        # Saturation
        plt.figure(figsize=(20, 3))
        histr = cv.calcHist([image], [1], None, [256], [0, 256])
        plt.xlim([0, 256])
        x = list(range(0, 256))
        colours = [colors.hsv_to_rgb((0, i / 256, 1)) for i in range(0, 256)]

        for i in x:
            plt.bar(x[i], histr[i], color=colours[i], edgecolor=colours[i], width=1)
            # plt.bar(range(0, 256), histr, color=colours, edgecolor=colours, width=1)
            plt.title('Saturation')
        plt.savefig(self.path_BarPlotCh2 + '\\BAR_CHART_SATURATION_' + name)
        # Value
        plt.figure(figsize=(20, 3))
        histr = cv.calcHist([image], [2], None, [256], [0, 256])
        plt.xlim([0, 256])
        x = list(range(0, 256))
        colours = [colors.hsv_to_rgb((0, 1, i / 256)) for i in range(0, 256)]
        for i in (x):
            plt.bar(x[i], histr[i], color=colours[i], edgecolor=colours[i], width=1)
            # plt.bar(range(0, 256), histr, color=colours, edgecolor=colours, width=1)
            plt.title('Value')
        plt.savefig(self.path_BarPlotCh3 + '\\BAR_CHART_VALUE_' + name)
        plt.close('all')

    def show_mask(self, mask, name):
        namefoldersplit = str.split(name, '.')
        namefolder = namefoldersplit[0]
        plt.figure(figsize=(10, 10))
        plt.imshow(mask, cmap='gray')
        plt.savefig(self.path_Mask + '\\MASK_' + name)
        plt.close('all')

    def overlay_mask(self, mask, image, name):
        namefoldersplit = str.split(name, '.')
        namefolder = namefoldersplit[0]
        rgb_mask = cv.cvtColor(mask, cv.COLOR_GRAY2RGB)
        img = cv.addWeighted(rgb_mask, 0.5, image, 0.5, 0)
        plt.imshow(img)
        plt.savefig(self.path_Mask_Overlay + '\\MASKOVERLAY_' + name)
        plt.close('all')

    def blurImage(self, image, name):
        image_blur = cv.GaussianBlur(image, (7, 7), 0)
        image_blur_hsv = cv.cvtColor(image_blur, cv.COLOR_RGB2HSV)
        min_red = np.array([10, 0, 0])
        max_red = np.array([40, 255, 255])
        image_red1 = cv.inRange(image_blur_hsv, min_red, max_red)
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 15))
        image_red_closed = cv.morphologyEx(image_red1, cv.MORPH_CLOSE, kernel)
        # Remove specks
        image_red_closed_then_opened = cv.morphologyEx(image_red_closed, cv.MORPH_OPEN, kernel)
        return name, image_red_closed_then_opened

    def findBiggestContour(self, image, name):
        # Copy to prevent modification
        image = image.copy()
        contours, hierarchy = cv.findContours(image, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        # Isolate largest contour
        # print(name + ":" + str(max(contours, key=cv.contourArea)))
        biggest_contour = max(contours, key=cv.contourArea)
        # Draw just largest contour
        mask = np.zeros(image.shape, np.uint8)
        cv.drawContours(mask, [biggest_contour], -1, 255, -1)
        return mask


# **5-Finally Run the main.py and try it**

In [ ]:
!python main.py

PreProcessing Directory Already Exists.
FeatureExtraction Directory Already Exists.
Sampling Directory Already Exists.
Classification Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Already Exists.
HSV Directory Already Exists.
LAB Directory Already Exists.
Lab_ Directory Already Exists.
ResizeImages Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Al

In [ ]:
#this just to know how to solve the Value ERrro in a simple version
from sklearn.model_selection import KFold
# Define the whole dataset
AllData = ["V","X", "Y", "T", "R", "A", "O", "Q", "N", "U", "G","P"]

# Define the training dataset (70% of AllData)
porData = ["Y", "T", "R", "A", "O", "Q", "N", "U"]

# Get the indices of the training dataset in the whole dataset
porDataIndex = [AllData.index(element) for element in porData if element in AllData]
print("AllData:",AllData)
print("portData :",porData)
print("indes all:",porDataIndex)

# Define the K-fold object with K=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Split the data into training and testing sets
for train_index, test_index in kf.split(porDataIndex):
    train_features=[]
    test_features=[]

    print("Training indices:", [porDataIndex[i] for i in train_index])
    print("Testing indices:", [porDataIndex[i] for i in test_index])

    for i in train_index:
        current=porData[i]
        train_features.append(porDataIndex[i])
        print(f"image with index ={porDataIndex[i]} and name {current} is added to training")

    for i in test_index:
        current=porData[i]
        test_features.append(porDataIndex[i])
        print(f"image with index ={porDataIndex[i]} and name {current} is added to test")

# **6-Test the model using images(not working) **

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.image import extract_patches_2d

def load_test_data(filename):
    df = pd.read_csv(filename)
    images = []
    for img in df['image']:
        img = Image.open(img)
        img = img.resize((224, 224))
        img = np.array(img)
        images.append(img)
    return np.array(images)

def load_model(filename):
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    return model

def preprocess_data(data):
    data = [img / 255.0 for img in data]  # Normalize pixel values
    data = [np.array(img) for img in data]  # Convert to numpy array
    return data

# Example usage:
test_data = load_test_data('test_data.csv')
model = load_model('trained_model.pkl')
preprocessed_data = preprocess_data(test_data)

# Load the saved model
model = load_model('trained_model.pkl')

# Prepare the test data
test_data = load_test_data('test_data.csv')

# Preprocess the test data
test_data = preprocess_data(test_data)

# Make predictions
predictions = model.predict(test_data)

# Evaluate the model
accuracy = accuracy_score(test_data['hue'], predictions)
print(f'Accuracy: {accuracy:.3f}')

# Visualize the results
import matplotlib.pyplot as plt

plt.scatter(test_data['hue'], predictions)
plt.xlabel('Actual Hue')
plt.ylabel('Predicted Hue')
plt.title('Model Performance')
plt.show()